In [4]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [6]:
instit = pd.read_excel('Estadísticas_artículos_científicos.xlsx', sheet_name='Institutions')
instit

,Institutions,records,%_of_14.717
0,UNIVERSITY OF LONDON,160,1.087
1,HARVARD UNIVERSITY,119,0.809
2,UNIVERSITY OF CALIFORNIA SYSTEM,102,0.693
3,HUAZHONG UNIVERSITY OF SCIENCE TECHNOLOGY,90,0.612
4,HARVARD MEDICAL SCHOOL,66,0.448
...,...,...,...
6312,ZUCKER SCH MED HOFSTRA NORTHWELL HLTH,1,0.007
6313,ZUNYI MED COLL AFFILIATED HOSP,1,0.007
6314,ZURICH UNIV HOSP,1,0.007
6315,ZUYDERLAND MED CTR,1,0.007


In [7]:
#Take the names of the institutions and put them on a list
direcciones = instit['Institutions'].to_list()

In [10]:
direcciones[0]

'UNIVERSITY OF LONDON'

In [13]:
#We tested first with a single direction
locator = Nominatim (user_agent = "yourmail") 
location = locator.geocode(direcciones[0])
print("Latitud = {}, Longitud = {}".format(location.latitude, location.longitude))

Latitud = 51.5229593, Longitud = -0.1291229


In [23]:
#We create a new list where the locations will be saved
localizaciones = []
 
#Each geolocation service such as Google Maps, Bing Maps, Yahoo, MapQuest or Nominatim has its own class in geopy.geocoders to use the API service.
#Create an object called a geolocation from the Nominatim class.
locator = Nominatim(user_agent = "yourEmail")#For a massive consultation you can put an email

#For each address stored in the 'addresses' list, we ask nominatim's geocoding services to return its coordinate to us and save it in the variable 'coordinates'. We add latitude and lo9ngitude to the 'locations' list.
for val in direcciones[0:10]:
    try:
        location = locator.geocode([val], timeout=15)
        localizaciones.append((location.latitude, location.longitude))
    except:
        localizaciones.append((np.NaN, np.NaN))
        continue
localizaciones

[(51.5229593, -0.1291229),
 (42.36790855, -71.12678237443698),
 (32.8745719, -117.23169004482253),
 (30.51547525, 114.40786094837449),
 (42.33597, -71.1011537201035),
 (51.5244443, -0.13351161008505222),
 (30.5398018, 114.35991954187581),
 (nan, nan),
 (43.663461999999996, -79.39775965337452),
 (48.11920765, -1.696073977992143)]

In [24]:
instit['Institutions'][7]#This one is failing, which we will have to look for with another procedure

'CHINESE ACADEMY OF MEDICAL SCIENCES PEKING UNION MEDICAL COLLEGE'

In [27]:
#Create a function that automates it to pass it through lambda to the df later
def geosearch(val):
    try:
        location = locator.geocode([val], timeout=15)
        lat = location.latitude
        long = location.longitude
    except:
        lat = np.NaN
        long = np.NaN
    return lat, long   

In [17]:
geosearch(direcciones[0])

(51.5229593, -0.1291229)

In [28]:
geosearch(direcciones[7])

(nan, nan)

In [29]:
instit['coord'] = instit['Institutions'].apply(lambda x: geosearch(x))

In [43]:
len(instit[instit.coord == (np.nan, np.nan)])#Half or so, we haven't gotten coordinates

3434

In [44]:
instit.to_csv('Instituciones_localizacion.csv')

In [121]:
instit = pd.read_csv('Instituciones_localizacion.csv', index_col=0)

In [100]:
instit['coord'].str.split(',')[0][0].replace('(', '')

'51.5229593'

In [89]:
#We tested the way to separate the two values into two columns
instit['coord'].str.split(',')[0][0].replace('(', '')

'51.5229593'

In [118]:
instit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6317 entries, 0 to 6316
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Institutions  6317 non-null   object 
 1   records       6317 non-null   int64  
 2   %_of_14.717   6317 non-null   float64
 3   coord         6317 non-null   object 
 4   Latitud       6317 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 616.1+ KB


In [122]:
#Convert to string the value of the coordinate field to extract the values to two columns
instit['coord'] = instit['coord'].apply(lambda x: str(x)).replace('b(','').replace(')','')

In [134]:
#We extract the coordinates to two columns
instit['Latitud'] = instit['coord'].apply(lambda x: str(x).split(',')[0].replace('(','')).astype(float)
instit['Longitud'] = instit['coord'].apply(lambda x: str(x).split(',')[1].replace(')','')).astype(float)

In [138]:
instit = instit.drop('coord', 1)
instit.head()

,Institutions,records,%_of_14.717,Latitud,Longitud
0,UNIVERSITY OF LONDON,160,1.087,51.522959,-0.129123
1,HARVARD UNIVERSITY,119,0.809,42.367909,-71.126782
2,UNIVERSITY OF CALIFORNIA SYSTEM,102,0.693,32.874572,-117.231690
3,HUAZHONG UNIVERSITY OF SCIENCE TECHNOLOGY,90,0.612,30.515475,114.407861
4,HARVARD MEDICAL SCHOOL,66,0.448,42.335970,-71.101154


In [139]:
#Save the CSV
instit.to_csv('Instituciones_localizacion_clean.csv')

In [146]:
#We extract the coordinates with Nan because he didn't find them with the above method
insti_nan = instit[instit.Latitud.isnull()]

In [166]:
insti_nan.Institutions

7       CHINESE ACADEMY OF MEDICAL SCIENCES PEKING UNI...
14                ASSISTANCE PUBLIQUE HOPITAUX PARIS APHP
25                              HUAZHONG UNIV SCI TECHNOL
26            SHAHID BEHESHTI UNIVERSITY MEDICAL SCIENCES
28      PENNSYLVANIA COMMONWEALTH SYSTEM OF HIGHER EDU...
                              ...                        
6310                   ZOO PROPHYLACT INST SOUTHERN ITALY
6311                           ZRN RHEINLAND ZRN MVZ GMBH
6312                ZUCKER SCH MED HOFSTRA NORTHWELL HLTH
6313                       ZUNYI MED COLL AFFILIATED HOSP
6315                                   ZUYDERLAND MED CTR
Name: Institutions, Length: 3434, dtype: object

In [154]:
#Percentage of results not found
len(insti_nan)/len(instit) *100 
#insti_nan

54.36124742757637

In [158]:
#We pobamos with different geocoders e.g. Yandex (Although it requires APIKEY)
#and gives us a free limit of 25,000 queries
#https://geopy.readthedocs.io/en/stable/#yandex
from geopy.geocoders import Yandex

In [176]:
#We tested first with the same address as With Nominated() to check the veracity of the geocoder
apikey = 'ded711c3-7320-4cfa-9275-ee4be8f2c707'
#To get the free apikey you can go to https://developer.tech.yandex.ru/
locator2 = Yandex (api_key=apikey, user_agent="jlfdez@yandex.ru") 
location2 = locator.geocode(direcciones[0])
print("Latitud = {}, Longitud = {}".format(location2.latitude, location2.longitude))
#It responds with the same coordinates so it seems reliable

Latitud = 51.5229593, Longitud = -0.1291229


In [178]:
#We re-created a function to go through the df
#We create a function that automates it to pass it through lambda to the df later
def geosearch_yandex(val, apikey):
    locator = Yandex (api_key=apikey, user_agent="jlfdez@yandex.ru") 
    try:
        location = locator.geocode([val], timeout=10)
        lat = location.latitude
        long = location.longitude
    except:
        lat = np.NaN
        long = np.NaN
    return lat, long

In [179]:
geosearch_yandex(direcciones[0], apikey)

(51.458902, 0.211913)

In [181]:
geosearch_yandex(insti_nan.Institutions[7], apikey)

(nan, nan)

In [182]:
insti_nan['coord'] = insti_nan['Institutions'].apply(lambda x: geosearch_yandex(x, apikey))

<ipython-input-182-73adfe54247e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insti_nan['coord'] = insti_nan['Institutions'].apply(lambda x: geosearch_yandex(x, apikey))


In [189]:
len(insti_nan[insti_nan['coord'] != (np.nan, np.nan)])
#We have achieved another 1224 institutions and their coordinates ()

1224

In [191]:
len(instit) #6,317 institutions in total (6,317 - 2,210 x 4,107 institutions with coordinates)
len(insti_nan)#3.434 - 1.224 = 2.210

3434